# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [19]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [21]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ghat,24.9647,10.1728,27.17,18,0,1.32,LY,2024-10-25 18:58:54+00:00
1,1,port-aux-francais,-49.3500,70.2167,2.25,67,48,10.32,TF,2024-10-25 18:53:09+00:00
2,2,olonkinbyen,70.9221,-8.7187,-1.56,64,100,8.59,SJ,2024-10-25 18:54:10+00:00
3,3,taoudenni,22.6783,-3.9836,29.20,22,0,3.79,ML,2024-10-25 19:00:03+00:00
4,4,kerikeri,-35.2268,173.9474,18.28,93,100,4.47,NZ,2024-10-25 19:00:04+00:00


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [24]:
%%capture --no-display

# Configure the map plot
city_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriTerrain",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)
# Display the map
city_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [27]:
# Narrow down cities that fit criteria and drop any results with null values
# Favorite temp criteria: 
#    max temperature lower than 27 degrees but higher than 21 
#    Wind speed less than 4.5 m/s
#    Zero cloudiness
favorite_citydata = city_data_df
favorite_citydata = favorite_citydata.loc[(favorite_citydata["Max Temp"]>21)&
                                            (favorite_citydata["Max Temp"]<27)&
                                            (favorite_citydata["Wind Speed"]<4.5)&
                                            (favorite_citydata["Cloudiness"]==0)]
# Drop any rows with null values
favorite_citydata = favorite_citydata.dropna()

# Display sample data
favorite_citydata.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,laguna,38.4210,-121.4238,21.79,41,0,2.57,US,2024-10-25 18:51:11+00:00
43,43,wailua homesteads,22.0669,-159.3780,25.44,87,0,3.09,US,2024-10-25 18:53:48+00:00
44,44,inhambane,-23.8650,35.3833,23.80,91,0,2.46,MZ,2024-10-25 19:00:52+00:00
147,147,upington,-28.4478,21.2561,26.21,8,0,2.06,ZA,2024-10-25 19:03:01+00:00
160,160,lichinga,-13.3128,35.2406,23.05,40,0,3.60,MZ,2024-10-25 19:03:18+00:00
191,191,dahab,28.5010,34.5134,26.55,28,0,3.24,EG,2024-10-25 19:03:54+00:00
249,249,fort bragg,35.1390,-79.0060,26.62,35,0,3.09,US,2024-10-25 19:05:03+00:00
318,318,salinas,36.6777,-121.6555,23.22,42,0,2.57,US,2024-10-25 19:05:02+00:00
359,359,middelburg,-25.7751,29.4648,23.71,35,0,4.30,ZA,2024-10-25 19:07:20+00:00
368,368,i-n-salah,27.1935,2.4607,26.85,25,0,2.36,DZ,2024-10-25 19:07:31+00:00


### Step 3: Create a new DataFrame called `hotel_df`.

In [30]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = favorite_citydata[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head(20)

,City,Country,Lat,Lng,Humidity,Hotel Name
11,laguna,US,38.4210,-121.4238,41,
43,wailua homesteads,US,22.0669,-159.3780,87,
44,inhambane,MZ,-23.8650,35.3833,91,
147,upington,ZA,-28.4478,21.2561,8,
160,lichinga,MZ,-13.3128,35.2406,40,
191,dahab,EG,28.5010,34.5134,28,
249,fort bragg,US,35.1390,-79.0060,35,
318,salinas,US,36.6777,-121.6555,42,
359,middelburg,ZA,-25.7751,29.4648,35,
368,i-n-salah,DZ,27.1935,2.4607,25,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [54]:
# Set parameters to search for a hotel 
# For each city, use the Geoapify API to find the first hotel located within 10,000 meters of your coordinates.
radius = 10000
params = {  
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel",
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat=row["Lat"]
    lng=row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Print the request URL for debugging
    #print(f"Request URL: {base_url}?{params}")
    
    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
   
    # Convert the API response to JSON format
    #name_address = name_address.json()
   
    # Check if the request was successful
    if name_address.status_code == 200:
        name_address = name_address.json()
        #print(f"Response: {name_address}")  # Debugging response
    
        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        try:
            hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        except (KeyError, IndexError):
            # If no hotel is found, set the hotel name as "No hotel found".
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    else:
        print(f"Error: {response.status_code} - {response.text}")
        hotel_df.loc[index, "Hotel Name"] = "Error in request"
         
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
laguna - nearest hotel: Holiday Inn Express & Suites
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
inhambane - nearest hotel: Hotel de Inhambane
upington - nearest hotel: Die Eiland Holiday Ressort
lichinga - nearest hotel: Residencial 2+1
dahab - nearest hotel: Lighthouse Hotel
fort bragg - nearest hotel: Airborne Inn Lodging
salinas - nearest hotel: The Traveler's Hotel
middelburg - nearest hotel: No hotel found
i-n-salah - nearest hotel: باجودة
seaside - nearest hotel: Embassy Suites
holualoa - nearest hotel: Kona Hotel
masvingo - nearest hotel: Hotel Victoria
reggane - nearest hotel: No hotel found
imbituba - nearest hotel: Silvestre Praia Hotel
rio grande - nearest hotel: Hotel Vila Moura Executivo
pacific grove - nearest hotel: Pacific Grove Inn
baddomalhi - nearest hotel: No hotel found
winton - nearest hotel: No hotel found
cidreira - nearest hotel: Hotel Castelo


,City,Country,Lat,Lng,Humidity,Hotel Name
11,laguna,US,38.4210,-121.4238,41,Holiday Inn Express & Suites
43,wailua homesteads,US,22.0669,-159.3780,87,Hilton Garden Inn Kauai Wailua Bay
44,inhambane,MZ,-23.8650,35.3833,91,Hotel de Inhambane
147,upington,ZA,-28.4478,21.2561,8,Die Eiland Holiday Ressort
160,lichinga,MZ,-13.3128,35.2406,40,Residencial 2+1
191,dahab,EG,28.5010,34.5134,28,Lighthouse Hotel
249,fort bragg,US,35.1390,-79.0060,35,Airborne Inn Lodging
318,salinas,US,36.6777,-121.6555,42,The Traveler's Hotel
359,middelburg,ZA,-25.7751,29.4648,35,No hotel found
368,i-n-salah,DZ,27.1935,2.4607,25,باجودة


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [57]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriTerrain",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)
# Display the map
hotel_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)